Often, data about the same individuals is maintained in more than one table. For example, one university office might have data about each student's time to completion of degree, while another has data about the student's tuition and financial aid.

To understand the students' experience, it may be helpful to put the two datasets together. If the data are in two tables, each with one row per student, then we would want to put the columns together, making sure to match the rows so that each student's information remains on a single row.

Let us do this in the context of a simple example, and then use the method with a larger dataset.

The table cones is one we have encountered earlier. Now suppose each flavor of ice cream comes with a rating that is in a separate table.

In [1]:
import pandas as pd
import numpy as np

In [35]:
cones = pd.DataFrame({
    'Flavor': ('strawberry', 'vanilla', 'chocolate', 'strawberry', 'chocolate'),
    'Price': (3.55, 4.75, 6.55, 5.25, 5.75)
})
cones

,Flavor,Price
0,strawberry,3.55
1,vanilla,4.75
2,chocolate,6.55
3,strawberry,5.25
4,chocolate,5.75


In [36]:
ratings = pd.DataFrame({
    'Kind': ('strawberry', 'chocolate', 'vanilla'),
    'Stars': (2.5, 3.5, 4)    
})
ratings

,Kind,Stars
0,strawberry,2.5
1,chocolate,3.5
2,vanilla,4.0


Each of the tables has a column that contains ice cream flavors: cones has the column Flavor, and ratings has the column Kind. The entries in these columns can be used to link the two tables.

The method join creates a new table in which each cone in the cones table is augmented with the Stars information in the ratings table. For each cone in cones, join finds a row in ratings whose Kind matches the cone's Flavor. We have to tell join to use those columns for matching.

In [37]:
rated = cones.join(ratings.set_index('Kind'), on='Flavor', how='inner').reset_index(drop=True)
rated

,Flavor,Price,Stars
0,strawberry,3.55,2.5
1,strawberry,5.25,2.5
2,vanilla,4.75,4.0
3,chocolate,6.55,3.5
4,chocolate,5.75,3.5


Each cone now has not only its price but also the rating of its flavor.

In general, a call to join that augments a table (say table1) with information from another table (say table2) looks like this:

In [39]:
rated['$/Star'] = rated['Price'] / rated['Stars']
rated = rated.sort_values('$/Star').reset_index(drop=True)
rated

,Flavor,Price,Stars,$/Star
0,vanilla,4.75,4.0,1.187500
1,strawberry,3.55,2.5,1.420000
2,chocolate,5.75,3.5,1.642857
3,chocolate,6.55,3.5,1.871429
4,strawberry,5.25,2.5,2.100000


In [40]:
ratings.join(cones.set_index('Flavor'), on='Kind', how='inner').reset_index(drop=True)

,Kind,Stars,Price
0,strawberry,2.5,3.55
1,strawberry,2.5,5.25
2,chocolate,3.5,6.55
3,chocolate,3.5,5.75
4,vanilla,4.0,4.75


Though strawberry has the lowest rating among the three flavors, the less expensive strawberry cone does well on this measure because it doesn't cost a lot per star.

Side note. Does the order we list the two tables matter? Let's try it. As you see it, this changes the order that the columns appear in, and can potentially changes the order of the rows, but it doesn't make any fundamental difference

Also note that the join will only contain information about items that appear in both tables. Let's see an example. Suppose there is a table of reviews of some ice cream cones, and we have found the average review for each flavor.

In [31]:
reviews = pd.DataFrame({
    'Flavor': ('vanilla', 'chocolate', 'vanilla', 'chocolate'),
    'Stars': (5, 3, 5, 4)    
})
reviews

,Flavor,Stars
0,vanilla,5
1,chocolate,3
2,vanilla,5
3,chocolate,4


In [27]:
average_review = reviews.groupby('Flavor')[['Stars']].mean().reset_index()
average_review

,Flavor,Stars
0,chocolate,3.5
1,vanilla,5.0


We can join cones and average_review by providing the labels of the columns by which to join.

In [41]:
cones.join(average_review.set_index('Flavor'), on='Flavor', how='inner')

,Flavor,Price,Stars
1,vanilla,4.75,5.0
2,chocolate,6.55,3.5
4,chocolate,5.75,3.5
